In [28]:
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

In [35]:
def prepare_img(image_path):
    # print(image_path)
    img = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
    # print(img.size)
    img = cv2.resize(img, (150, 150))
    img = img / 255.0
    return img

def load_data(path):
    data = []
    file_names = os.listdir(path)
    # print(file_names)
    for name in file_names:
        data.append(prepare_img(os.path.join(path,name)))
    


    return np.array(data)


In [81]:
# model = models.Sequential([
#     layers.Conv2D(32, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(64, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Conv2D(128, (3, 3), activation='relu'),
#     layers.MaxPooling2D((2, 2)),
#     layers.Flatten(),
#     layers.Dense(512, activation='relu'),
#     layers.Dropout(0.5),
#     layers.Dense(1, activation='sigmoid')

# ])


classifier = models.Sequential()
classifier.add(layers.Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))
classifier.add(layers.MaxPooling2D(pool_size = (2, 2)))
classifier.add(layers.Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(layers.MaxPooling2D(pool_size=(2, 2)))
classifier.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(layers.MaxPooling2D(pool_size=(2, 2)))
classifier.add(layers.Flatten())
classifier.add(layers.Dense(units = 64, activation = 'relu'))
classifier.add(layers.Dropout(0.5))
classifier.add(layers.Dense(1))
classifier.add(layers.Activation('sigmoid'))

classifier.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

c:\Users\klonl\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [84]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory("C:\\Users\\klonl\\Kosciuszkon\\archive\\Plastic Bottle Image Dataset\\train\\",
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = "binary")
test_set = test_datagen.flow_from_directory("C:\\Users\\klonl\\Kosciuszkon\\archive\\Plastic Bottle Image Dataset\\test\\",
                                                 target_size = (150, 150),
                                                 batch_size = 32,
                                                 class_mode = "binary")


Found 3194 images belonging to 2 classes.
Found 3469 images belonging to 2 classes.


In [85]:
history = classifier.fit(training_set, epochs=10)

Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 49s 454ms/step - accuracy: 0.5916 - loss: 10.2493
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 41s 393ms/step - accuracy: 0.7411 - loss: 0.5538
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 42s 391ms/step - accuracy: 0.7723 - loss: 0.4941
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 42s 397ms/step - accuracy: 0.7928 - loss: 0.4381
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 42s 399ms/step - accuracy: 0.8285 - loss: 0.4169
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 43s 405ms/step - accuracy: 0.8654 - loss: 0.3738
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 44s 414ms/step - accuracy: 0.8860 - loss: 0.3321
Epoch 8/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 45s 422ms/step - accuracy: 0.8860 - loss: 0.3457
Epoch 9/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 43s 410ms/step - accuracy: 0.9003 - loss: 0.3013
Epoch 10/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 43s 406ms/step - accuracy: 0.8978 - loss: 0.2889


In [106]:
test_im = keras.preprocessing.image.load_img("C:\\Users\\klonl\\OneDrive\\Obrazy\\Z aparatu\\puchav3.jpg",target_size=(150,150))
test_im = keras.preprocessing.image.img_to_array(test_im)
test_im = np.expand_dims(test_im, axis = 0)
print("Butelka" if classifier.predict(test_im)[0,0] == 1 else "Nie butelka")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Nie butelka


In [107]:
classifier.save('exported_model.keras')